we join demographics data from the statewide database, to the LA_times votes+precincts data.

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np

In [2]:
import maup

Change the block data epsg to 4326 to have it in the same coordinates system as the precincts data.

In [3]:
blocks = gpd.read_file('../CA_state_Data/Demographics_swdb/blocks_latlng.shp')

In [4]:
precincts = gpd.read_file('../CA_state_Data/CA_fixed_vf/CA_fixed_vf.shp')

In [5]:
blocks.crs, precincts.crs

({'init': 'epsg:4326'}, {'init': 'epsg:4326'})

In [6]:
blocks.columns.to_list()

['STATEFP10',
 'COUNTYFP10',
 'tract_x',
 'block',
 'geoid',
 'NAME10',
 'MTFCC10',
 'UR10',
 'UACE10',
 'UATYPE',
 'FUNCSTAT10',
 'ALAND10',
 'AWATER10',
 'INTPTLAT10',
 'INTPTLON10',
 'TOTPOP',
 'NH_WHITE',
 'NH_BLACK',
 'NH_AMIN',
 'NH_ASIAN',
 'NH_NHPI',
 'NH_OTHER',
 'NH_2MORE',
 'HISP',
 'VAP',
 'HVAP',
 'WVAP',
 'BVAP',
 'AMINVAP',
 'ASIANVAP',
 'NHPIVAP',
 'OTHERVAP',
 '2MOREVAP',
 'NAME',
 'state',
 'county',
 'tract_y',
 'geometry']

In [7]:
variables = ["TOTPOP", 
 'NH_WHITE',
 'NH_BLACK',
 'NH_AMIN',
 'NH_ASIAN',
 'NH_NHPI',
 'NH_OTHER',
 'NH_2MORE',
 'HISP',
 'VAP',
 'HVAP',
 'WVAP',
 'BVAP',
 'AMINVAP',
 'ASIANVAP',
 'NHPIVAP',
 'OTHERVAP',
 '2MOREVAP']


In [8]:
from maup.assign import assign_by_covering, assign_to_max

# read in blocks and precincts GeoDataFrames

inters = maup.intersections(blocks, precincts)


In [10]:
source_assignment = pd.Series(
    inters.index.get_level_values(0), index=inters.index
)
coverage = inters.area / source_assignment.map(blocks.area)
assignment = assign_to_max(coverage)

In [11]:

precincts[variables] = blocks[variables].groupby(assignment).sum()
precincts[variables].head()

,TOTPOP,NH_WHITE,NH_BLACK,NH_AMIN,NH_ASIAN,NH_NHPI,NH_OTHER,NH_2MORE,HISP,VAP,HVAP,WVAP,BVAP,AMINVAP,ASIANVAP,NHPIVAP,OTHERVAP,2MOREVAP
0,2223.0,208.0,387.0,15.0,470.0,0.0,6.0,23.0,1114.0,1637.0,761.0,182.0,296.0,14.0,362.0,0.0,4.0,18.0
1,1270.0,707.0,61.0,2.0,371.0,7.0,3.0,42.0,77.0,1195.0,69.0,672.0,61.0,2.0,348.0,6.0,3.0,34.0
2,3378.0,72.0,732.0,3.0,66.0,15.0,1.0,41.0,2448.0,2297.0,1570.0,64.0,578.0,2.0,47.0,11.0,0.0,25.0
3,1700.0,114.0,854.0,1.0,230.0,2.0,7.0,39.0,453.0,1312.0,303.0,107.0,682.0,1.0,189.0,2.0,5.0,23.0
4,1755.0,1015.0,67.0,3.0,389.0,1.0,10.0,94.0,176.0,1616.0,163.0,954.0,42.0,3.0,363.0,1.0,8.0,82.0


In [14]:
precincts.to_file('CA_votes_dem_pct.shp')

In [12]:
precincts.to_csv('joined_dem.csv')

In [13]:
precincts.to_pickle('joined_dem.pkl')